<a href="https://colab.research.google.com/github/rhemish/rhemish/blob/main/MLF_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Machine Learning Frameworks Final Project**

**Predicting Stock Prices Using Machine Learning**

In [1]:
import pandas as pd

# Setting chunk size for stock_prices
chunksize = 100000

# Processing stock prices data in chunks
all_stock_prices = []
for chunk in pd.read_csv('/content/stock_prices_latest.csv', chunksize=chunksize, parse_dates=['date'] if 'date' in pd.read_csv('/content/stock_prices_latest.csv', nrows=1).columns else [0], index_col=0 if 'date' not in pd.read_csv('/content/stock_prices_latest.csv', nrows=1).columns else None, engine='python'):
    all_stock_prices.append(chunk)
stock_prices = pd.concat(all_stock_prices)

# Loading dividends and earnings data directly (no chunking)
dividends = pd.read_csv('/content/dividends_latest.csv', parse_dates=['date'] if 'date' in pd.read_csv('/content/dividends_latest.csv', nrows=1).columns else [0], index_col=0 if 'date' not in pd.read_csv('/content/dividends_latest.csv', nrows=1).columns else None, engine='python')
earnings = pd.read_csv('/content/earnings_latest.csv', parse_dates=['date'] if 'date' in pd.read_csv('/content/earnings_latest.csv', nrows=1).columns else [0], index_col=0 if 'date' not in pd.read_csv('/content/earnings_latest.csv', nrows=1).columns else None, engine='python')


In [2]:
print("Dividends Data:")
print(dividends.head())

print("\nEarnings Data:")
print(earnings.head())

print("\nStock Prices Data:")
print(stock_prices.head())


Dividends Data:
  symbol       date  dividend
0   MSFT 2016-11-15      0.39
1   MSFT 2011-05-17      0.16
2   MSFT 2008-05-13      0.11
3   MSFT 2011-02-15      0.16
4   MSFT 2012-02-14      0.20

Earnings Data:
  symbol       date      qtr  eps_est  eps release_time
0      A 2009-05-14  04/2009      NaN  NaN         post
1      A 2009-08-17  07/2009      NaN  NaN         post
2      A 2009-11-13  10/2009      NaN  NaN          pre
3      A 2010-02-12  01/2010      NaN  NaN          pre
4      A 2010-05-17  04/2010      NaN  NaN         post

Stock Prices Data:
  symbol       date   open   high    low  close  close_adjusted     volume  \
0   MSFT 2016-05-16  50.80  51.96  50.75  51.83         49.7013   20032017   
1   MSFT 2002-01-16  68.85  69.84  67.85  67.87         22.5902   30977700   
2   MSFT 2001-09-18  53.41  55.00  53.17  54.32         18.0802   41591300   
3   MSFT 2007-10-26  36.01  36.03  34.56  35.03         27.2232  288121200   
4   MSFT 2014-06-27  41.61  42.29  41.51  

In [3]:
# Merging stock prices with dividends on a common column
merged_data = pd.merge(stock_prices, dividends, on=['symbol', 'date'], how='left')

# Merging with earnings data
merged_data = pd.merge(merged_data, earnings, on=['symbol', 'date'], how='left')

print(merged_data.head())

  symbol       date   open   high    low  close  close_adjusted     volume  \
0   MSFT 2016-05-16  50.80  51.96  50.75  51.83         49.7013   20032017   
1   MSFT 2002-01-16  68.85  69.84  67.85  67.87         22.5902   30977700   
2   MSFT 2001-09-18  53.41  55.00  53.17  54.32         18.0802   41591300   
3   MSFT 2007-10-26  36.01  36.03  34.56  35.03         27.2232  288121200   
4   MSFT 2014-06-27  41.61  42.29  41.51  42.25         38.6773   74640000   

   split_coefficient  dividend  qtr  eps_est  eps release_time  
0                1.0       NaN  NaN      NaN  NaN          NaN  
1                1.0       NaN  NaN      NaN  NaN          NaN  
2                1.0       NaN  NaN      NaN  NaN          NaN  
3                1.0       NaN  NaN      NaN  NaN          NaN  
4                1.0       NaN  NaN      NaN  NaN          NaN  


In [4]:
merged_data.columns

Index(['symbol', 'date', 'open', 'high', 'low', 'close', 'close_adjusted',
       'volume', 'split_coefficient', 'dividend', 'qtr', 'eps_est', 'eps',
       'release_time'],
      dtype='object')

In [5]:
merged_data = merged_data[['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'eps_est', 'eps']]

In [6]:
pip install requests pandas

In [7]:
import requests
import pandas as pd

# Fetching news data from the API
url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&apikey=M9HROIEL6ZIJXKPZ'
response = requests.get(url)
news_data = response.json()

# Converting articles to DataFrame
articles = news_data['feed']
news_df = pd.DataFrame(articles)

# Exploding the 'ticker_sentiment' column
news_df = news_df.explode('ticker_sentiment')

# Normalizing the 'ticker_sentiment' column to extract relevant ticker information
ticker_sentiments = pd.json_normalize(news_df['ticker_sentiment'])

# Concatenating the ticker data with the rest of the DataFrame
news_df = pd.concat([news_df.reset_index(drop=True), ticker_sentiments.reset_index(drop=True)], axis=1)

# Selecting relevant columns for aggregation
news_df_reduced = news_df[['time_published', 'ticker', 'overall_sentiment_label']]

# Converting 'time_published' to datetime for easier aggregation
news_df_reduced['time_published'] = pd.to_datetime(news_df_reduced['time_published'])

# Aggregating by date and ticker
aggregated_sentiments = news_df_reduced.groupby(
    [news_df_reduced['time_published'].dt.date, 'ticker']
).agg({
    'overall_sentiment_label': lambda x: x.mode()[0] if not x.empty else 'neutral'  # Most common sentiment
}).reset_index()

# Renaming columns for clarity
aggregated_sentiments.rename(columns={'time_published': 'date'}, inplace=True)

# Displaying the aggregated sentiments
print(aggregated_sentiments.head())

         date ticker overall_sentiment_label
0  2025-04-14    ACN                 Bullish
1  2025-04-14   AMZN                 Neutral
2  2025-04-14    APG        Somewhat-Bullish
3  2025-04-14   APOS                 Neutral
4  2025-04-14   APTV        Somewhat-Bullish


<ipython-input-7-1b7756ad931c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df_reduced['time_published'] = pd.to_datetime(news_df_reduced['time_published'])


In [8]:
# Defining the mapping for sentiment labels
sentiment_mapping = {
    'neutral': 0,
    'somewhat-bullish': 1,
    'bullish': 2,
    'somewhat-bearish': -1,
    'bearish': -2
}

# Applying the mapping to create a new column
aggregated_sentiments['sentiment_score'] = aggregated_sentiments['overall_sentiment_label'].map(sentiment_mapping)

# Dropping the text label
aggregated_sentiments.drop(columns=['overall_sentiment_label'], inplace=True)

print(aggregated_sentiments.head())

         date ticker  sentiment_score
0  2025-04-14    ACN              NaN
1  2025-04-14   AMZN              NaN
2  2025-04-14    APG              NaN
3  2025-04-14   APOS              NaN
4  2025-04-14   APTV              NaN


In [9]:
# Renaming and aligning columns in aggregated_sentiments for merging
aggregated_sentiments.rename(columns={'ticker': 'symbol', 'time_published': 'date'}, inplace=True)

# Converting 'date' columns to datetime format
merged_data['date'] = pd.to_datetime(merged_data['date'])
aggregated_sentiments['date'] = pd.to_datetime(aggregated_sentiments['date'])

In [10]:
merged_data.shape

(24395016, 10)

In [11]:
# Breaking merged_data into smaller chunks
chunk_size = 10000
chunks = [merged_data[i:i + chunk_size] for i in range(0, merged_data.shape[0], chunk_size)]

processed_chunks = []

# Process each chunk
for chunk in chunks:
    chunk['date'] = pd.to_datetime(chunk['date'], errors='coerce')
    aggregated_sentiments['date'] = pd.to_datetime(aggregated_sentiments['date'], errors='coerce')

    # Perform the merge
    chunk_result = chunk.merge(
        aggregated_sentiments[['symbol', 'date', 'sentiment_score']],
        on=['date', 'symbol'],  # Using 'date' and 'symbol' as keys for merging
        how='left'  # Using 'left' join to keep all rows from merged_data
    )

    processed_chunks.append(chunk_result)

# Combining all processed chunks into a single DataFrame
final_merged_data = pd.concat(processed_chunks, ignore_index=True)

print(final_merged_data.head())

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['date'] = pd.to_datetime(chunk['date'], errors='coerce')  # Convert 'date' in merged_data
<ipython-input-11-128e2a8fa263>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['date'] = pd.to_datetime(chunk['date'], errors='coerce')  # Convert 'date' in merged_data
<ipython-input-11-128e2a8fa263>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

  symbol       date   open   high    low  close     volume  dividend  eps_est  \
0   MSFT 2016-05-16  50.80  51.96  50.75  51.83   20032017       NaN      NaN   
1   MSFT 2002-01-16  68.85  69.84  67.85  67.87   30977700       NaN      NaN   
2   MSFT 2001-09-18  53.41  55.00  53.17  54.32   41591300       NaN      NaN   
3   MSFT 2007-10-26  36.01  36.03  34.56  35.03  288121200       NaN      NaN   
4   MSFT 2014-06-27  41.61  42.29  41.51  42.25   74640000       NaN      NaN   

   eps  sentiment_score  
0  NaN              NaN  
1  NaN              NaN  
2  NaN              NaN  
3  NaN              NaN  
4  NaN              NaN  


In [14]:
final_merged_data['sentiment_score'] = final_merged_data['sentiment_score'].fillna(0)

In [15]:
print(final_merged_data.info())
print(final_merged_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24395016 entries, 0 to 24395015
Data columns (total 11 columns):
 #   Column           Dtype         
---  ------           -----         
 0   symbol           object        
 1   date             datetime64[ns]
 2   open             float64       
 3   high             float64       
 4   low              float64       
 5   close            float64       
 6   volume           int64         
 7   dividend         float64       
 8   eps_est          float64       
 9   eps              float64       
 10  sentiment_score  float64       
dtypes: datetime64[ns](1), float64(8), int64(1), object(1)
memory usage: 2.0+ GB
None
  symbol       date   open   high    low  close     volume  dividend  eps_est  \
0   MSFT 2016-05-16  50.80  51.96  50.75  51.83   20032017       NaN      NaN   
1   MSFT 2002-01-16  68.85  69.84  67.85  67.87   30977700       NaN      NaN   
2   MSFT 2001-09-18  53.41  55.00  53.17  54.32   41591300       NaN      N

In [16]:
for i, chunk in enumerate(processed_chunks):
    chunk.to_csv(f'processed_chunk_{i}.csv', index=False)

In [17]:
final_merged_data.columns

Index(['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividend',
       'eps_est', 'eps', 'sentiment_score'],
      dtype='object')

In [ ]:
print(final_merged_data.describe())